In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = set(i.split(".csv")[0] for i in os.listdir("./cases/fluxes/nonIBD/") if "csv" in i)

In [4]:
len(organisms)

495

### Load

In [5]:
def load_data(path):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

In [6]:
from tqdm import tqdm
conditions = ["nonIBD", "IBD"]

In [7]:
# overflow = {i:pandas.DataFrame() for i in conditions}
growth = {i:{} for i in conditions}

In [8]:
# for org in tqdm(organisms):
#     for c in conditions:
#         tmp = load_data("./cases/overflow/{}/{}.csv".format(c,org))
#         overflow[c] = pandas.concat([overflow[c],tmp], axis=1).fillna(0)

In [9]:
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/fluxes/{}red/{}.csv".format(c,org))
        if tmp.empty:
            val = None
        else:
            val = tmp[org]["biomass_dilution"]
        growth[c][org] = val
growth = pandas.DataFrame.from_dict(growth)

100%|██████████| 495/495 [00:03<00:00, 134.66it/s]


In [10]:
growth.dropna()

,nonIBD,IBD
Collinsella_intestinalis_DSM_13280,0.001125,0.001125
Campylobacter_lari_RM2100,0.001161,0.001452
Faecalibacterium_cf_prausnitzii_KLE1255,0.089407,0.089407
Bacteroides_sp_D1,0.001217,0.097225
Acidaminococcus_fermentans_DSM_20731,0.001000,0.001001
...,...,...
Brevibacillus_borstelensis_AK1,0.001002,0.001000
Eubacterium_siraeum_DSM_15702,0.001074,0.043164
Paraprevotella_xylaniphila_YIT_11841,0.001136,0.001751
Aeromonas_veronii_B565,0.001019,0.001023


### Save

In [11]:
growth.sort_index().to_csv("datasets/3.5.IBDred_growth_rates.csv")